## 1.0 Importing Libraries

In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import os
import pandas as pd
import openpyxl
import time
import re
import requests
import numpy as np



## 2.0 Defining Paths

In [8]:
driver_path = r"C:\Users\MSIKa\Downloads\Python-Projects\chromedriver-win64"
output_folder_path = r"C:\Users\MSIKa\Downloads\Python-Projects\Automation Projects\01 Selenium_BeautifulSoup\02_Genre_wise_Excels" 
homepage_url = r"https://books.toscrape.com/"

## 3.0 Defining functions

### 3.1 General functions

In [9]:
#Creating a log file creation function-> if not exist- then we create an empty df, if exists- then we load the excel as a df
def log_file_creation(output_folder_path):
    folder_list = os.listdir(output_folder_path)

    if "Log_File.xlsx" not in folder_list:
        log_file = pd.DataFrame()
        # log_file["Genre"] = []
        log_file["Genre"] = ""
        log_file["Page_No"] = ""
        log_file["Status"] = ""
    else:
        log_file = pd.read_excel(os.path.join(output_folder_path, "Log_File.xlsx"))

    return log_file




#Finally creating the consolidate output
def Consolidated_Output(output_folder_path, genre_name):
    with pd.ExcelWriter(os.path.join(output_folder_path, "Consolidated_Output.xlsx"), mode = "w", engine = "openpyxl") as writer:
        for i in range(0, len(genre_name)):
            #Going thorugh each genre's folder and creating list of files
            genre_excel_path = os.path.join(output_folder_path, f"{i+1}_{genre_name[i]}")
            
            original_excel_list = os.listdir(genre_excel_path)
            updated_excel_list = []
    
            #If atleast one file exists then we proceed else skip the curr genre
            if original_excel_list:
                #As temp files are also created so need to remove them and created another empty string
                for file in original_excel_list:
                    if file.endswith(".xlsx") and not file.startswith("~"):
                        updated_excel_list.append(file)
    
                if updated_excel_list:
                    genre_excel_df = pd.read_excel(os.path.join(genre_excel_path, updated_excel_list[0]))
                    genre_excel_df.to_excel(writer, sheet_name = genre_name[i], index= False)
            else:
                continue


def create_and_open_driver(driver_path, homepage_url):
    #Initialising the Web driver 
    service = Service(ChromeDriverManager().install())
    options = Options()
    options.add_argument("start-maximized")
    
    driver = webdriver.Chrome(service=service, options=options)
        
    #Wait here
    driver.get(homepage_url)

    return driver

### 3.2 Traversal functions

In [10]:
#Defining a function for iterating over each of the books of a page
def book_function(Rating_dict, book_img_element, books_on_page, book_no, page_no, row_no, worksheet, driver):
    Price = books_on_page[book_no].select_one("div.product_price p.price_color").text.strip()
    Price = float(Price.strip("£"))
    
    Availability = books_on_page[book_no].select_one("div.product_price p.instock.availability").text.strip()

    #title is always returned as a string
    Title = books_on_page[book_no].select_one("h3 a")["title"]

    #class is always returned as a list, converting the rating to numeric
    Star_Rating = books_on_page[book_no].select_one("p.star-rating")["class"][1]
    Rating = Rating_dict[Star_Rating]

    

    #Fetching the books description 

    #Clicking on the book's img to get to book page
    driver.execute_script("arguments[0].scrollIntoView();", book_img_element[book_no])
    driver.execute_script("arguments[0].click();", book_img_element[book_no])


    #finding the desc element
    try:
        desc_element = driver.find_element(By.CSS_SELECTOR, "div#product_description + p")
        desc = desc_element.text.strip()
    except NoSuchElementException:
        desc="NA"

    #Getting the inventory count
    inventory_count_element = driver.find_element(By.CSS_SELECTOR, "div.row p.instock.availability").text.strip()
    inventory_count = re.search(r"\d+", inventory_count_element).group(0)

    
    #Updating the excel workbook with book's info
    worksheet.cell(row = row_no+book_no, column = 1, value = Title)
    worksheet.cell(row = row_no+book_no, column = 2, value = Price)
    worksheet.cell(row = row_no+book_no, column = 3, value = Rating)
    worksheet.cell(row = row_no+book_no, column = 4, value = Availability)
    worksheet.cell(row = row_no+book_no, column = 5, value = inventory_count)
    worksheet.cell(row = row_no+book_no, column = 6, value = page_no)
    worksheet.cell(row = row_no+book_no, column = 7, value = desc)

    
    #Getting back to genre-specific page and to avoid stale reference error we update the book elements list
    driver.back()
    html_code = driver.page_source
    soup = BeautifulSoup(html_code, "html.parser")
    books_on_page = soup.select("ol.row li article.product_pod")
    book_img_element = driver.find_elements(By.CSS_SELECTOR, "img.thumbnail")

    return books_on_page, book_img_element, worksheet
    
    


#Defining a function for each page of a specific genre
def page_function(workbook_path, output_folder_path, Rating_dict, genre_name, curr_genre, driver, log_file, page_no, completed_till_page_no):

    #This should only run for records that already existed and only the first time when page_no = completed_till_page_no
    if page_no == completed_till_page_no and completed_till_page_no != 0 :
        # next_button = driver.find_element(By.CSS_SELECTOR, "ul.pager li.next a")
        for count in range(0, page_no):
            next_button = driver.find_element(By.CSS_SELECTOR, "ul.pager li.next a")
            driver.execute_script("arguments[0].scrollIntoView();", next_button)
            driver.execute_script("arguments[0].click();", next_button)
            time.sleep(3)
    
    #Initializing parsed_html_code to get the html source of the page
    html_code = driver.page_source
    soup = BeautifulSoup(html_code, "html.parser")

    #Finding all the book elements on the page
    books_on_page = soup.select("ol.row li article.product_pod")

    #Creating img element list
    book_img_element = driver.find_elements(By.CSS_SELECTOR, "img.thumbnail")


    #Loading the workbook for each page instead of each book
    workbook = openpyxl.load_workbook(os.path.join(workbook_path, f"{genre_name[curr_genre]}_Books.xlsx"))
    worksheet = workbook.active
    row_no = worksheet.max_row+1
    page_no = page_no+1
    
    for book_no in range(0, len(books_on_page)):
        books_on_page, book_img_element, worksheet = book_function(Rating_dict, book_img_element, books_on_page, book_no, page_no, row_no, worksheet, driver)
        
    #Saving each genre excel 
    workbook.save(os.path.join(workbook_path, f"{genre_name[curr_genre]}_Books.xlsx"))

    #Updating the log file's page number and saving it 
    log_file.loc[log_file["Genre"] == genre_name[curr_genre], "Page_No"] = page_no
    log_file.to_excel(os.path.join(output_folder_path, "Log_File.xlsx"), index= False)
    
    #finding and clicking on next button
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, "ul.pager li.next a")
        #Wait here
        driver.execute_script("arguments[0].scrollIntoView();", next_button)
        driver.execute_script("arguments[0].click();", next_button)

        #Recursive calling
        page_function(workbook_path, output_folder_path, Rating_dict, genre_name, curr_genre, driver, log_file, page_no, completed_till_page_no)

    except NoSuchElementException:
        #Updating the log file's completion status and saving it
        log_file.loc[log_file["Genre"] == genre_name[curr_genre], "Status"] = "Complete"
        log_file.to_excel(os.path.join(output_folder_path, "Log_File.xlsx"), index= False)
        return
    



#Defining a function for running on each genre

def genre_function(output_folder_path, log_file, Rating_dict, genre_name, curr_genre, driver_path, homepage_url, completed_till_page_no):
    #Initialising the Web driver 
    driver = create_and_open_driver(driver_path, homepage_url)
    
    #Creating list of the genre elements
    book_genre = driver.find_elements(By.CSS_SELECTOR, "ul.nav.nav-list li ul li a")
    
    #Finding specific genre one-at-a-time and clicking it
    driver.execute_script("arguments[0].scrollIntoView();", book_genre[curr_genre])
    driver.execute_script("arguments[0].click();", book_genre[curr_genre])

    
    #Creating folder path for saving genre-wise excels
    workbook_path = os.path.join(output_folder_path, f"{curr_genre+1}_{genre_name[curr_genre]}")

    if not os.path.exists(os.path.join(workbook_path, f"{genre_name[curr_genre]}_Books.xlsx")):
        #Initialize Excel for containing the Book info
        workbook = openpyxl.Workbook()
        worksheet = workbook.active
        worksheet.cell(row=1, column = 1, value="Title")
        worksheet.cell(row=1, column = 2, value="Price(£)")
        worksheet.cell(row=1, column = 3, value="Star Ratings")
        worksheet.cell(row=1, column = 4, value="Availability")
        worksheet.cell(row=1, column = 5, value="Inventory Count")
        worksheet.cell(row=1, column = 6, value="Page No")
        worksheet.cell(row=1, column = 7, value="Description")
    
        #saving workbook
        workbook.save(os.path.join(workbook_path, f"{genre_name[curr_genre]}_Books.xlsx"))

        #Initializing the page no
        page_no = 0

    else:
        page_no = completed_till_page_no

    log_file.loc[curr_genre, "Genre"] = genre_name[curr_genre]
    page_function(workbook_path, output_folder_path, Rating_dict, genre_name, curr_genre, driver, log_file, page_no, completed_till_page_no)



    driver.quit()

    
    # #Going to homepage and refreshing the genre_list to avoid the stale element error
    # driver.get(homepage_url)
    # #Wait here
    # book_genre = driver.find_elements(By.CSS_SELECTOR, "ul.nav.nav-list li ul li a")
    # return book_genre


## 4.0 Orchestrator

In [11]:
#Creating list of the genre elements
response = requests.get(homepage_url)
html_code = response.text

soup = BeautifulSoup(html_code, "html.parser")
book_genre = soup.select("ul.nav.nav-list li ul li a")

#Creating a list for containing genre element's name
genre_name = []
for i in range(0, len(book_genre)):
    genre_name.append(book_genre[i].text.strip())


#Creating a folder in Downloads
os.makedirs(output_folder_path, exist_ok=True)

#Creating first separate folders for housing each genre's excel
for curr_genre in range(0, len(genre_name)):
    os.makedirs(os.path.join(output_folder_path, f"{curr_genre+1}_{genre_name[curr_genre]}"), exist_ok = True)
    
#Creating a rating dict to convert aplhabetical ratings to numerical
Rating_dict = {"One":1, "Two":2, "Three":3, "Four":4, "Five":5}

#Calling log file creation function if log file not exist- then we create an empty df, if exists- then we load the excel as a df
log_file = log_file_creation(output_folder_path)


for curr_genre in range(0, len(genre_name)):  

    #Genre in log file
    if genre_name[curr_genre] in log_file["Genre"].tolist():
        if log_file.loc[log_file["Genre"]==genre_name[curr_genre], "Status"].iloc[0] == "Complete":
            continue

        else:
            try:
                completed_till_page_no = log_file.loc[log_file["Genre"]==genre_name[curr_genre], "Page_No"].iloc[0]
                genre_function(output_folder_path, log_file, Rating_dict, genre_name, curr_genre, driver_path, homepage_url, completed_till_page_no)
            except Exception as e:
                print(f"Fetching of {genre_name[curr_genre]} Books has Failed\n")

            
    #Genre not in log file 
    else:    
        try:
            completed_till_page_no = 0
            genre_function(output_folder_path, log_file, Rating_dict, genre_name, curr_genre, driver_path, homepage_url, completed_till_page_no)
        except Exception as e:
            print(f"Fetching of {genre_name[curr_genre]} Books has Failed\n")




#Consolidating all the individual genre excels
Consolidated_Output(output_folder_path, genre_name)